In [3]:
import layout
import sys
import random
import os
from QLearningAgent import *
from search import *
from transitionFunction import *
from pacman import ClassicGameRules
import noise
import json
from pathlib import Path

In [2]:
def default(str):
    return str + ' [Default: %default]'


In [3]:
def defineAgents(agentOpts, pacmanName, noKeyboard):

    pacmanType = loadAgent(pacmanName, noKeyboard)
    pacman = pacmanType(agentOpts["pacman"])

    ghost_list = []
    for ghost in agentOpts["ghosts"]:
        ghostType = loadAgent(ghost["name"], 1)
        ghost_list.append(
            ghostType(index=ghost["args"]["index"], prob=ghost["args"]["prob"]))

    return pacman, ghost_list

In [4]:

"""
@param argv
read the commandline and set the paramator of the pacman game
"""
def readCommand(argv):
    """
    Processes the command used to run pacman from the command line.
    """
    from optparse import OptionParser
    usageStr = """
    USAGE:      python pacman.py <options>
    EXAMPLES:   (1) python pacman.py
                    - starts an interactive game
                (2) python pacman.py --layout smallClassic --zoom 2
                OR  python pacman.py -l smallClassic -z 2
                    - starts an interactive game on a smaller board, zoomed in
    """
    parser = OptionParser(usageStr)
    parser.add_option('-m', '--mode', dest='mode',
                      help=default(
                          'type of statistics you want to run (s, e, g, t)'),
                      default='s')
    parser.add_option('-l', '--layout', dest='layout',
                      help=default(
                          'the LAYOUT_FILE from which to load the map layout'),
                      metavar='LAYOUT_FILE', default='mediumClassic')
    parser.add_option('-p', '--pacman', dest='pacman',
                      help=default(
                          'the agent TYPE in the pacmanAgents module to use'),
                      metavar='TYPE', default='KeyboardAgent')
    parser.add_option('-g', '--ghosts', dest='ghost',
                      help=default(
                          'the ghost agent TYPE in the ghostAgents module to use'),
                      metavar='TYPE', default='RandomGhost')
    parser.add_option('-k', '--numghosts', type='int', dest='numGhosts',
                      help=default('The maximum number of ghosts to use'), default=4)
    parser.add_option('-z', '--zoom', type='float', dest='zoom',
                      help=default('Zoom the size of the graphics window'), default=1.0)
    parser.add_option('-q', '--quietTextGraphics', action='store_true', dest='quietGraphics',
                      help='Generate minimal output and no graphics', default=False)
    parser.add_option('-r', '--recording', action='store_true', dest='recordingParam',
                      help='Generate recordings of testing epochs', default=False)
    parser.add_option('-f', '--fixRandomSeed', action='store_true', dest='fixRandomSeed',
                      help='Fixes the random seed to always play the same game', default=False)
    parser.add_option('-a', '--agentArgs', dest='agentArgs',
                      help='Comma separated values sent to agent. e.g. "opt1=val1,opt2,opt3=val3"')
    parser.add_option('--timeout', dest='timeout', type='int',
                      help=default('Maximum length of time an agent can spend computing in a single game'), default=30)
    parser.add_option('-s', '--statArgs', dest='statArgs',
                      help='Comma separated values sent to stat. e.g. "opt1=val1,opt2,opt3=val3"')
    parser.add_option('-c', '--swapsArg', dest='swapsArg',
                      help='Comma separated values sent to stat. e.g. "opt1=val1,opt2,opt3=val3"')
    parser.add_option('-t', '--pretrainedAgent', dest='pretrainedAgent', help=default('The pre-trained pacmanAgents module to use'),
                      metavar='TYPE', default=None)
    parser.add_option('-o', '--outputStats', dest='outputStats', help=default('Output file to save stats'),
                      metavar='TYPE', default='saved_agent.pkl')
    parser.add_option('-d', '--savedfFolder', dest='savedfFolder', help=default('Output file to save stats'),
                      metavar='TYPE', default='.')
    parser.add_option('-n', '--noiseArgs', dest='noiseArgs',
                      help='Comma separated values sent to noise. e.g. "opt1=val1,opt2,opt3=val3,noise=\{arg1=val1,arg2=val2\}"')
    parser.add_option('-y', '--ghostArgs', dest='noiseArgs',
                      help='Comma separated values sent to noise. e.g. "opt1=val1,opt2,opt3=val3,noise=\{arg1=val1,arg2=val2\}"')
    parser.add_option('--frameTime', dest='frameTime', type='float',
                      help=default('Time to delay between frames; <0 means keyboard'), default=0.1)

    options, otherjunk = parser.parse_args(argv)
    if len(otherjunk) != 0:
        raise Exception('Command line input not understood: ' + str(otherjunk))
    args = dict()

    # Fix the random seed
    if options.fixRandomSeed:
        random.seed('cs188')

    # Choose a layout
    args['layout'] = layout.getLayout(options.layout)
    if args['layout'] == None:
        raise Exception("The layout " + options.layout + " cannot be found")

    # Choose a Pacman agent
    noKeyboard = options.quietGraphics

    print(options.agentArgs)

    try:
        agentOpts = json.loads(options.agentArgs)
    except:
        agentOpts = {}

    try:
        args['statOpts'] = json.loads(options.statArgs)
    except:
        args['statOpts'] = {}
    
    print(args['statOpts'])

    args["ghosts"] = {}
    args['pacman'] = {}
    args["perturbOpts"] = {}

    # Instantiate Pacman with agentArgs
    pacman, ghosts = defineAgents(
        agentOpts["test"], options.pacman, noKeyboard)
    args['pacman']["test"] = pacman
    args["ghosts"]["test"] = []
    args["ghosts"]["test"] = ghosts
    args["perturbOpts"]["test"] = agentOpts["test"]["perturb"]

    if "ensemble" in agentOpts:
        pacman, ghosts = defineAgents(
            agentOpts["test"], options.pacman, noKeyboard)
        args['pacman']["ensemble"] = pacman
        args["ghosts"]["ensemble"] = []
        args["ghosts"]["ensemble"] = ghosts
        args["perturbOpts"]["ensemble"] = agentOpts["ensemble"]["perturb"]

    # Don't display training games
    if 'numTrain' in agentOpts:
        options.numQuiet = int(agentOpts['numTrain'])
        options.numIgnore = int(agentOpts['numTrain'])

    if 'savedFolder' in agentOpts:
        args['savedFolder'] = options.savedFolder

    # Choose a display format
    if options.quietGraphics:
        import textDisplay
        args['display'] = textDisplay.NullGraphics()
    else:
        import graphicsDisplay
        args['display'] = graphicsDisplay.PacmanGraphics(
            options.zoom, frameTime=options.frameTime)

    if options.recordingParam:
        args['recording'] = options.outputStats + "-RECORDING-"
    else:
        args['recording'] = None

    args['timeout'] = options.timeout
    args['agentOpts'] = agentOpts
    args['pacmanAgentName'] = options.pacman
    args['pretrainedAgentName'] = options.pretrainedAgent
    args['outputStats'] = options.outputStats
    args['swapsArg'] = options.swapsArg

    args['mode'] = options.mode

    return args


In [5]:
GENERALIZATION_WORLDS = [{"pacman": {}, "ghosts": [{"name": "RandomGhost", "args": {
    "index": 1, "prob": {}}}], "perturb": {"noise": {"mean": 0, "std": 0.1}, "perm": {}}},{"pacman": {}, "ghosts": [{"name": "RandomGhost", "args": {
    "index": 1, "prob": {}}}], "perturb": {"noise": {"mean": 0, "std": 0.2}, "perm": {}}},{"pacman": {}, "ghosts": [{"name": "RandomGhost", "args": {
    "index": 1, "prob": {}}}], "perturb": {"noise": {"mean": 0, "std": 0.3}, "perm": {}}},{"pacman": {}, "ghosts": [{"name": "RandomGhost", "args": {
    "index": 1, "prob": {}}}], "perturb": {"noise": {"mean": 0, "std": 0.5}, "perm": {}}},{"pacman": {}, "ghosts": [{"name": "RandomGhost", "args": {
    "index": 1, "prob": {}}}], "perturb": {"noise": {"mean": 0, "std": 0.7}, "perm": {}}},{"pacman": {}, "ghosts": [{"name": "RandomGhost", "args": {
    "index": 1, "prob": {}}}], "perturb": {"noise": {"mean": 0, "std": 0.9}, "perm": {}}}]
#This contains basically everything about the pacman game

In [6]:
SWAP_LIST = [0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9]

In [7]:
def recordRange(index, range):
    if index < range["max"] and index >= range["min"]: 
        return True
    else:
        return False

In [8]:
record_range = None
if record_range:
    print("yay")

### runLearnability (func)

In [5]:
pacman =pacman
testingenv_mean=0
testingenv_std=0
testingenv_noise_args='{"mean":'0',"std":'0'}'
pacmanArgs = '{"test":{"pacman":{},"ghosts":'{"index":1,"prob":0.1}',"perturb":'{"noise":'$testingenv_noise_args',"perm":{}}'

ghosts
layout = $(echo "$line" | cut -d "," -f 1)
display = 
file_to_be_loaded=None
applyperturb=None
record=None
epochs=1000
trained_agents=500
n_training_steps=10
n_testing_steps=10
record_range=None
run_untill=None
timeout=30

record_range =  {'max': 10, 'min': 0}
epochs = 10
training_agents = 500
graphicsDisplay.PacmanGraphics(
            options.zoom, frameTime=options.frameTime)


SyntaxError: unterminated string literal (detected at line 5) (2404019454.py, line 5)

In [7]:
stats = np.zeros(
        [500, 100 // 10], dtype=np.float32)
print(stats)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
if not os.path.exists(args['outputStats'].split('/')[0]):
        os.makedirs(args['outputStats'].split('/')[0])